# FAA 1462 Pareja 10. Práctica 4 REGRESIÓN LOGÍSTICA Y ANÁLISIS ROC
## Kevin de la Coba Malam

En este documento se hara un análisis del clasificador de **Algoritmos Genéticos** usando diferenrtes *poblaciones*, *generaciones*, *cruces*, *mutaciones* y *elitismo*. También se comparará el algoritmo con el clasificador de  **Naive Bayes** mediante el *espacio ROC*. 

En está practica, se ha usado solo la validación simple. En concreto, con la validación simple se usán valores para el %: **20** (se ejecutará 3 veces). Al tener varios tipos de mutación y cruce el test se hace muy largo.

Para simplificar el documento se ha creado el archivo *utils.py* en el cual se encuentran las funciones para clasificar, mostrar gráficas...

### Modificaciones

- La clase Datos se ha modificado, el constructor tiene un parametro opcional (allNominal) que por defecto está a False para tratar a todos los atributos como si fuesen nominales. 

In [4]:
# Para esconder los warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from ClasificadorAlgoritmoGenetico import AlgoritmoGenetico as AG
from Datos import Datos
import utils
import matplotlib.pyplot as plt

# Cargamos datos
l = Datos("lentillas.data")
titanic = Datos("titanic.csv", allNominal=True)
ttt = Datos("tic-tac-toe.data")

## 1. Detalles de la implementación

Lo primero sería mencionar la estructura que tiene cada individuo. Para crear esta estructura se ha hecho una clase interna en la cual se define a un individuo con los siguientes datos.
```py
class Individuo:
	fitnessValue # Valor de fitness del individuo
	priori # Clase mayoritaria de los datos
	reglas = [
		{"regla": [int], "conclusion": int} # Cada regla es un diccionario
		...
	]
```
Para representar las reglas se usa un diccionario, en este diccionario tenemos la clave "regla" en la cual el valor es una lista de enteros. Cada entero de esta lista hace referencia a la cadena de bits para un atributo, por ejemplo: Si tenemos 3 atributos A, B y C, tendremos 3 enteros en la lista, si cada atributo tiene 3 valores, se usaran 3 bits de cada entero. Para operar con estos bits se usan bit operators (&, |).

La conclusión es un entero que marca 0 o 1.

### 1.1 Generación de la población inicial

Las reglas se crean de forma aleatoria, asignando para cada atributo una cadena aleatoria (de la cual solo se van a usar los N primeros bits, siendo N el número de posibles valores en el atributo). Para este análisis se usaran individuos con un **máximo de 5 reglas**.

### 1.2 Cruces implementados

Se han implementado el cruce *inter reglas* y el cruce *intra reglas*. Son métodos estáticos de la clase *ClasificadorAlgoritmoGenetico* que deben pasarse al propio constructor de la clase (patrón template).

### 1.3 Mutaciones implementadas

Se ha implementado la *mutación estandar* y la *mutación de reglas*. De nuevo, son métodos estáticos que deben pasarse al constructor.

### 1.4 Clasificación

La clasificación es simple, se coge al mejor individuo del entrenamiento y se aplican las reglas, pero, ¿cómo se aplican estas reglas? Al utilizar cadenas de bits para cada atributo es simple comparar si un valor se cumple en una regla o no, por ejemplo:
Si tenemos el atributo A con los posibles datos 'a', 'b', 'c', tendríamos una cadena de bits para este atributo con 3 bits. Si recibimos el valor 'a', recibimos la cadena 001 (un entero con valor 1), si recibimos 'b' 010 (entero con valor 2) y si recibimos 'c' 100 (entero con valor 4). Con puertas lógicas es muy simple comparar estos valores. Si tenemos la regla 101 (entero con valor 5) y recibimos el valor 001 (a) (aplicamos la puerta lógica and sobre los valores enteros 5 y 1, 101 & 001), pues esa parte de la regla se aprobaría para este atributo y haría falta hacerlo con todos los demás, si se cumplen todos devolvemos la conclusión.

En el entrenamiento (también en la clasificación) puede pasar que varias relgas se cumplan o que no se cumpla ninguna, en el primer caso lo que se hace es devolver la conclusión mayoritaria, en el segundo caso se devulve la clase a priori (clase mayoritaria).

## 2. Resultados clasificación.

Las opciones que se han usado para la clasificación son las siguientes:
- **Población**: Variable [50-150].
- **Generacion**: Variable [100-200].
- **Máximo número de reglas**: 5.
- **Probabilidad mutación**: 5%.
- **Proporción de elitismo**: 5%.
- **Cruce**: Ambos (inter e intra).
- **Mutación**: Ambas (estandar y de reglas).

### Cruce *Inter Reglas* y mutación *estandar*

In [ ]:
errorMedioTTT_inter_estandar, errorMedioTITANIC_inter_estandar = utils.AG_test(ttt, titanic, AG.cruceInterReglas, AG.mutacionEstandar)

### Cruce *Inter Reglas* y mutación *de reglas*

In [ ]:
errorMedioTTT_inter_reglas, errorMedioTITANIC_inter_reglas = utils.AG_test(ttt, titanic, AG.cruceInterReglas, AG.mutacionReglas)

### Cruce *Intra Reglas* y mutación *estandar*

In [ ]:
errorMedioTTT_intra_estandar, errorMedioTITANIC_intra_estandar = utils.AG_test(ttt, titanic, AG.cruceIntraReglas, AG.mutacionEstandar)

### Cruce *Intra Reglas* y mutación *de reglas*

In [ ]:
errorMedioTTT_intra_reglas, errorMedioTITANIC_intra_reglas = utils.AG_test(ttt, titanic, AG.cruceIntraReglas, AG.mutacionReglas)

# PONER CONCLUSION APARTADO ANTERIOR. DECIR QUE poblacion y generaciones es mejor, y decir que cruce y que mutacion es mejor.

Tambien decir que en la validación se tiene en cuenta mucho el priori, ya que puede pasar que el mejor individuo no tenga tantas reglas y tenga que recurrir al priori. Que para mi sería mejor no solo guardar al mejor individuo, cuando el mejor individuo trate de devolver el priori, que en vez de eso vaya al segundo mejor. 

## 3. Análisis
Hablar de overfitting si ponemos muchas reglas. 

In [ ]:
# Ejemplo con 10 reglas y 20 individuos
ag = AG(20, 100, 10, AG.cruceInterReglas, AG.mutacionReglas , 0.05, 0.05, show=True)
ag.entrenamiento(titanic.datos, titanic.nominalAtributos, titanic.diccionario)

Hablar de la relación entre el poblacion y generaciones. Cuanta mas poblacion menos generaciones y cuantas mas generaciones menos poblacion.
Decir que si la mutacion es muy alta, las cosas se pueden ir de madre.



In [ ]:
# Ejemplo con 10 reglas y 20 individuos, probabilidad mutación 70%. No elitismo
ag = AG(10, 100, 5, AG.cruceInterReglas, AG.mutacionReglas , 0.7, 0.0, show=True)
ag.entrenamiento(titanic.datos, titanic.nominalAtributos, titanic.diccionario)

Hablar sobre el ejemplo con mutación muy alta.

## 4. Evolución del fitness
### 4.1 Titanic

In [ ]:
# Plot de 2 fitness maximo y medio, en cada plot varios runs
utils.plot(titanic, "Titanic", AG.cruceInterReglas, AG.mutacionReglas)

hablar de el crecimiento.

### 4.2 Tic-Tac-Toe

In [ ]:
# Plot de 2 fitness maximo y medio, en cada plot varios runs
utils.plot(ttt, "Tic-Tac-Toe", AG.cruceInterReglas, AG.mutacionReglas)

Hablar del crecimiento.

## 5. Espacio ROC 
### 5.1 Titanic

In [6]:
# Generar espacio ROC con naive bayes
utils.plot_espacio_ROC(titanic, 5, 20, AG.cruceInterReglas, AG.mutacionReglas)

KeyboardInterrupt: 

Conclusion